### 다중 클래스 분류 모델 추론 코드 
영화 댓글 데이터 감성 분석

In [25]:
# pytorch 프레임워크 라이브러리
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# 자연어 처리 라이브러리
from torch.nn.functional import softmax
from sklearn.model_selection import train_test_split
from transformers import AdamW, BertModel
from transformers.optimization import get_cosine_schedule_with_warmup
import gluonnlp as nlp

# 기타
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from datetime import datetime, timedelta
import os
import time
import json

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=0,  
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate    
        self.num_classes = num_classes
        self.classifier = nn.Linear(hidden_size , self.num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

def get_model_and_vocab(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
        vocab_file, padding_token="[PAD]"
    )
    return bertmodel, vocab_b_obj

# 모델 설정 함수
def setup_model(model_path, vocab_file, device, num_classes):
    bert_model, vocab = get_model_and_vocab(model_path, vocab_file)
    tokenizer = nlp.data.BERTSPTokenizer(vocab_file, vocab, lower=False)
    model = BERTClassifier(bert_model, num_classes=num_classes, dr_rate=0.5).to(device)
    return model, tokenizer


In [27]:
# 기본 파라미터 설정
max_len=256
batch_size=32
warmup_ratio=0.1
num_epochs=50
max_grad_norm=1
log_interval=100
learning_rate=5e-5
num_classes = 2 # 분류 할 클래스의 개수
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def main():
    # 모델 로드 및 데이터 준비
    base_model_path = '../kobert-base-v1'
    vocab_file = '../kobert-base-v1/kobert_news_wiki_ko_cased-1087f8699e.spiece'

    model, tokenizer = setup_model(base_model_path, vocab_file, device, num_classes)
    return model, tokenizer

model, tokenizer = main()

## 학습 모델 로드
model.load_state_dict(torch.load('./2024-03-31_06-17/movie-review-entiment-analysis-model.pt'))  # state_dict를 불러 온 후, 모델에 저장

<All keys matched successfully>

In [ ]:
model

In [45]:
def predict(sentence):
    def get_dataloader(sentence):
        data = [sentence, '0']
        dataset_another = [data]

        another_test = BERTDataset(dataset_another, 0, 1, tokenizer, max_len, True, False)
        dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
        return dataloader

    # 분류 프로세스 
    dataloader = get_dataloader(sentence)

    top_idx = 0
    top_logit = 0

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()
            probabilities = softmax(torch.from_numpy(logits), dim=0).numpy()
            top_prob_indices = np.argsort(probabilities)[::-1][:1]
            
            top_idx = top_prob_indices[0]
            top_logit = probabilities[top_idx]

    if top_idx == 0:
        print('부정')
    else:
        print('긍정')
    print(f'{round(top_logit*100, 2)}%')


In [54]:
predict('이야기를 끌고 나가는 힘이 부족함')

부정
86.56%
